In [1]:
import numpy as np
import math
import matplotlib
import matplotlib.pyplot as plt
from collections import Counter
import pickle
import pandas as pd
from sklearn.metrics import accuracy_score
import time

%matplotlib inline

## Conv Net

In [42]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import torchvision


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, 5, 1, padding=2)
        self.conv2 = nn.Conv2d(64, 32, 5, 1, padding=1)
        self.fc1 = nn.Linear(32 * 6 * 6, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 2)
        self.dropout = nn.Dropout(p=0.2)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 32 * 6 * 6)
        x = F.relu(self.fc1(self.dropout(x)))
        x = F.relu(self.fc2(self.dropout(x)))
        x = self.fc3(self.dropout(x))
        return F.log_softmax(x, dim=1)
    
def train(model, device, train_loader, optimizer, epoch, print_every):
    model.train()
    running_loss = 0.0
    total_train_loss = 0
    
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        # print statistics
        running_loss += loss.item()
        total_train_loss += loss.item()
        if (batch_idx + 1) % (print_every + 1) == 0:
            print('Train Epoch: {} [{:d}%] \tLoss: {:.2f}'.format(
                epoch, int(100. * (batch_idx + 1) / len(train_loader)), running_loss / print_every))
            running_loss = 0.0
    print("Train Epoch {} : Total Average loss {:.4f}".format(epoch , total_train_loss/(batch_idx+1)))

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f})\n'.format(
        test_loss, correct, len(test_loader.dataset),
        1. * correct / len(test_loader.dataset)))

## MNIST

In [2]:
with open('./data/mnist/mnist_X_train.p', 'rb') as f:
    X_train = pickle.load(f)
with open('./data/mnist/mnist_y_train.p', 'rb') as f:
    y_train = pickle.load(f)
with open('./data/mnist/mnist_X_test.p', 'rb') as f:
    X_test = pickle.load(f)
with open('./data/mnist/mnist_y_test.p', 'rb') as f:
    y_test = pickle.load(f)

In [3]:
class DatasetMNIST(Dataset):
    
    def __init__(self, X_path, y_path, transform=None):
        with open(X_path, 'rb') as f:
            self.X = pickle.load(f)
        with open(y_path, 'rb') as f:
            self.y = pickle.load(f)
        self.transform = transform
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        # load image as ndarray type (Height * Width * Channels)
        # be carefull for converting dtype to np.uint8 [Unsigned integer (0 to 255)]
        # in this example, i don't use ToTensor() method of torchvision.transforms
        # so you can convert numpy ndarray shape to tensor in PyTorch (H, W, C) --> (C, H, W)
        image = self.X[index].astype(np.uint8).reshape((28, 28, 1))
        label = self.y[index]
        
        if self.transform is not None:
            image = self.transform(image*255)
            label = torch.tensor(label, dtype = torch.long)
        return image, label

In [4]:
test_dataset = DatasetMNIST('./data/mnist/mnist_X_test.p','./data/mnist/binarized_mnist_y_test.p', transform=transforms.ToTensor())
image,label = test_dataset.__getitem__(0)

print('images shape on batch size = {}'.format(image.shape))
print('labels shape on batch size = {}'.format(label))

images shape on batch size = torch.Size([1, 28, 28])
labels shape on batch size = 1


In [12]:
use_cuda = torch.cuda.is_available()
#use_cuda = False
device = torch.device("cuda" if use_cuda else "cpu")

batch_size = 64
test_batch_size = 1000 
epochs = 6
save_model = False

lr = 0.01
momentum = 0.5

start = time.time()

kwargs = {'num_workers': 0, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    DatasetMNIST('./data/mnist/mnist_X_train.p', './data/mnist/binarized_mnist_y_train.p',
                    transform = transforms.ToTensor()),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    DatasetMNIST('./data/mnist/mnist_X_test.p', './data/mnist/binarized_mnist_y_test.p',
                    transform = transforms.ToTensor()),
    batch_size=test_batch_size, shuffle=True, **kwargs)

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch, log_interval = 10)
    test(model, device, test_loader)
    
test(model, device, train_loader)

end = time.time() - start
print("Time spent : {:4f}s".format(end))

if (save_model):
    torch.save(model.state_dict(),"mnist_cnn.pt")

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.695102
Train Epoch: 1 [640/60000 (1%)]	Loss: 0.693300
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.690600
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.694535
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.692658
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.691321
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.691916
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.691113
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.691524
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.692293
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.689471
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.688778
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.689689
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.687404
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.687675
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.687807
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.691165
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.685879
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.689223
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.681345
Train Epoch: 1 [12800/60000 (

Train Epoch: 4 [27520/60000 (46%)]	Loss: 0.026666
Train Epoch: 4 [28160/60000 (47%)]	Loss: 0.074507
Train Epoch: 4 [28800/60000 (48%)]	Loss: 0.083031
Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.032103
Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.021236
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.091910
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.084662
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.011986
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.025051
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.074098
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.214458
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.030044
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.068012
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.059280
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.120306
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.041344
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.021737
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.021560
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.039989
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.031672


## CIFAR

In [4]:
with open('./data/cifar/cifar_X_train.p', 'rb') as f:
    X_train = pickle.load(f)
with open('./data/cifar/binarized_cifar_y_train.p', 'rb') as f:
    y_train = pickle.load(f)
with open('./data/cifar/cifar_X_test.p', 'rb') as f:
    X_test = pickle.load(f)
with open('./data/cifar/binarized_cifar_y_test.p', 'rb') as f:
    y_test = pickle.load(f)

In [5]:
class DatasetCIFAR(Dataset):
    
    def __init__(self, X_path, y_path, transform=None):
        with open(X_path, 'rb') as f:
            self.X = pickle.load(f)
        with open(y_path, 'rb') as f:
            self.y = pickle.load(f)
        self.transform = transform
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        # load image as ndarray type (Height * Width * Channels)
        # be carefull for converting dtype to np.uint8 [Unsigned integer (0 to 255)]
        # in this example, i don't use ToTensor() method of torchvision.transforms
        # so you can convert numpy ndarray shape to tensor in PyTorch (H, W, C) --> (C, H, W)
        image = self.X[index].astype(np.uint8).reshape((32, 32, 3))
        label = self.y[index]
        
        if self.transform is not None:
            image = self.transform(image*255)
            label = torch.tensor(label, dtype = torch.long)
        return image, label

In [7]:
test_dataset = DatasetCIFAR('./data/cifar/cifar_X_test.p','./data/cifar/binarized_cifar_y_test.p', transform=transforms.ToTensor())
image,label = test_dataset.__getitem__(0)

print('images shape on batch size = {}'.format(image.shape))
print('labels shape on batch size = {}'.format(label))

images shape on batch size = torch.Size([3, 32, 32])
labels shape on batch size = 3


In [25]:
class Net2(nn.Module):
    def __init__(self):
        super(Net2, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 5, 1, padding=2)
        self.conv2 = nn.Conv2d(64, 32, 5, 1, padding=1)
        self.fc1 = nn.Linear(32 * 7 * 7, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 2)
        self.dropout = nn.Dropout(p=0.2)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 32 * 7 * 7)
        x = F.relu(self.fc1(self.dropout(x)))
        x = F.relu(self.fc2(self.dropout(x)))
        x = self.fc3(self.dropout(x))
        return F.log_softmax(x, dim=1)

In [44]:
use_cuda = torch.cuda.is_available()
#use_cuda = False
device = torch.device("cuda" if use_cuda else "cpu")

batch_size = 64
test_batch_size = 1000 
epochs = 40
save_model = True

lr = 0.01
momentum = 0.5

start = time.time()

kwargs = {'num_workers': 0, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    DatasetCIFAR('./data/cifar/cifar_X_train.p', './data/cifar/binarized_cifar_y_train.p',
                    transform = transforms.ToTensor()),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    DatasetCIFAR('./data/cifar/cifar_X_test.p', './data/cifar/binarized_cifar_y_test.p',
                    transform = transforms.ToTensor()),
    batch_size=test_batch_size, shuffle=True, **kwargs)

model = Net2().to(device)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch, 78)
    test(model, device, test_loader)
    
test(model, device, train_loader)

end = time.time() - start
print("Time spent : {:4f}s".format(end))

if (save_model):
    torch.save(model.state_dict(),"cifar_cnn.pt")

Train Epoch: 1 [10%] 	Loss: 0.70
Train Epoch: 1 [20%] 	Loss: 0.70
Train Epoch: 1 [30%] 	Loss: 0.70
Train Epoch: 1 [40%] 	Loss: 0.70
Train Epoch: 1 [50%] 	Loss: 0.70
Train Epoch: 1 [60%] 	Loss: 0.70
Train Epoch: 1 [70%] 	Loss: 0.70
Train Epoch: 1 [80%] 	Loss: 0.70
Train Epoch: 1 [90%] 	Loss: 0.70
Train Epoch 1 : Total Average loss 0.6904

Test set: Average loss: 0.6830, Accuracy: 5726/10000 (0.57)

Train Epoch: 2 [10%] 	Loss: 0.69
Train Epoch: 2 [20%] 	Loss: 0.69
Train Epoch: 2 [30%] 	Loss: 0.69
Train Epoch: 2 [40%] 	Loss: 0.69
Train Epoch: 2 [50%] 	Loss: 0.69
Train Epoch: 2 [60%] 	Loss: 0.68
Train Epoch: 2 [70%] 	Loss: 0.68
Train Epoch: 2 [80%] 	Loss: 0.68
Train Epoch: 2 [90%] 	Loss: 0.68
Train Epoch 2 : Total Average loss 0.6772

Test set: Average loss: 0.6604, Accuracy: 6101/10000 (0.61)

Train Epoch: 3 [10%] 	Loss: 0.67
Train Epoch: 3 [20%] 	Loss: 0.67
Train Epoch: 3 [30%] 	Loss: 0.67
Train Epoch: 3 [40%] 	Loss: 0.67
Train Epoch: 3 [50%] 	Loss: 0.67
Train Epoch: 3 [60%] 	Loss: 0.67


Train Epoch: 21 [20%] 	Loss: 0.47
Train Epoch: 21 [30%] 	Loss: 0.46
Train Epoch: 21 [40%] 	Loss: 0.47
Train Epoch: 21 [50%] 	Loss: 0.48
Train Epoch: 21 [60%] 	Loss: 0.46
Train Epoch: 21 [70%] 	Loss: 0.47
Train Epoch: 21 [80%] 	Loss: 0.48
Train Epoch: 21 [90%] 	Loss: 0.47
Train Epoch 21 : Total Average loss 0.4661

Test set: Average loss: 0.5893, Accuracy: 6969/10000 (0.70)

Train Epoch: 22 [10%] 	Loss: 0.43
Train Epoch: 22 [20%] 	Loss: 0.44
Train Epoch: 22 [30%] 	Loss: 0.45
Train Epoch: 22 [40%] 	Loss: 0.46
Train Epoch: 22 [50%] 	Loss: 0.45
Train Epoch: 22 [60%] 	Loss: 0.46
Train Epoch: 22 [70%] 	Loss: 0.47
Train Epoch: 22 [80%] 	Loss: 0.47
Train Epoch: 22 [90%] 	Loss: 0.48
Train Epoch 22 : Total Average loss 0.4533

Test set: Average loss: 0.6365, Accuracy: 6811/10000 (0.68)

Train Epoch: 23 [10%] 	Loss: 0.43
Train Epoch: 23 [20%] 	Loss: 0.43
Train Epoch: 23 [30%] 	Loss: 0.44
Train Epoch: 23 [40%] 	Loss: 0.44
Train Epoch: 23 [50%] 	Loss: 0.45
Train Epoch: 23 [60%] 	Loss: 0.44
Train Ep


Time spent : 1127.643266s
